# PyConES 2022: Chatbots, Reconocimiento de Voz y Text-to-Speech: Tu Asistente Virtual Open-Source 



## 1. Manejando el audio desde nuestro PC

Para manejar la grabación y la reproducción de audio en nuestro ordenador a través de Python podemos usar PyAudio.

| Nombre      | PyAudio                                                                                                                                                                                                                                                                             |
|-------------|-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| Licencia    | MIT                                                                                                                                                                                                                                                                                 |
| Descripción | (Traducido) PyAudio proporciona enlaces de Python para PortAudio v19, la biblioteca de E/S de audio multiplataforma. Con PyAudio, puedes utilizar fácilmente Python para reproducir y grabar audio en una variedad de plataformas, como GNU/Linux, Microsoft Windows y Apple macOS. |

Los archivos a grabar/reproducir usan el estándar WAVE para poder extraer toda la información.

### 1.1 Muestreo y precisión
En el mundo del audio digital, todo funciona por 1s y 0s. Pero el sonido es realmente analógico, por lo que hay que parametrizar el mundo analógico en datos discretos. De esta manera, necesitamos recoger con cierta frecuencia la información, y también alguna manera de representar la intensidad de la información del audio.

A la frecuencia de recogida se le conoce como **muestreo**.

A el nivel de detalle que podemos recoger la información del audio en digital se le conoce como **precisión** o **profundidad**. Cuantos más bits se requieran para tomar el dato, más profundidad tendrá.


![Muestreo](img/muestreo.png)
![Precisión](img/precision.png)


In [4]:
# Script de reproducción
import wave
import pyaudio

pyaudio_play = pyaudio.PyAudio()
archivo = wave.open('file.wav','rb')

stream = pyaudio_play.open(
    format = pyaudio_play.get_format_from_width(archivo.getsampwidth()),
    channels = archivo.getnchannels(),
    rate = archivo.getframerate(),
    output = True
)

datos = archivo.readframes(1024)

while len(datos) > 0:
        # writing to the stream is what *actually* plays the sound.
    stream.write(datos)
    datos = archivo.readframes(1024)

pyaudio_play.close(stream)
print("Ha acabado la reproducción")

Ha acabado la reproducción


In [2]:
# Script de grabación
from array import array
import pyaudio
import wave
 
FORMAT = pyaudio.paInt16
CHANNELS = 1
RATE = 16000
CHUNK = 1024
RECORD_SECONDS = 5
WAVE_OUTPUT_FILENAME = "file.wav"
THRESHOLD = 2000
 
audio = pyaudio.PyAudio()
audio_data = []
can_record = False
finish_record = False
silent_chunks = 0
 
# start Recording
stream = audio.open(format=FORMAT, channels=CHANNELS,
                rate=RATE, input=True,
                frames_per_buffer=CHUNK)
print ("recording...")
 

while not finish_record:
    chunk = stream.read(CHUNK)
    data_array = array('h', chunk)
    print(max(data_array))
    if not can_record:
        can_record = max(data_array) >= THRESHOLD
    else:
        audio_data.append(chunk)
        if (max(data_array) <= THRESHOLD):
            silent_chunks += 1
            print("Silencio {0}".format(silent_chunks))
        else:
            silent_chunks = 0
        if silent_chunks >= 10:
            finish_record = True
    


print ("finished recording")
 
 
# stop Recording
stream.stop_stream()
stream.close()
audio.terminate()
 
waveFile = wave.open(WAVE_OUTPUT_FILENAME, 'wb')
waveFile.setnchannels(CHANNELS)
waveFile.setsampwidth(audio.get_sample_size(FORMAT))
waveFile.setframerate(RATE)
waveFile.writeframes(b''.join(audio_data))
waveFile.close()

recording...
1
1
1
262
265
307
270
318
505
294
305
303
309
340
331
322
309
339
350
421
32516
32555
31479
29391
12730
5306
291
Silencio 1
220
Silencio 2
266
Silencio 3
32362
32546
22565
27203
32563
1870
Silencio 1
683
Silencio 2
24159
32677
31870
21462
5010
182
Silencio 1
169
Silencio 2
321
Silencio 3
271
Silencio 4
250
Silencio 5
360
Silencio 6
237
Silencio 7
229
Silencio 8
194
Silencio 9
179
Silencio 10
finished recording


## 2. Reconocimiento de voz. Convirtiendo el sonido en texto. 

El Reconocimiento de Voz o Automatic Speech Recognition (ASR) como la capacidad de un programa de procesar el habla y convertirlo en un texto escrito legible, que puedan entender también las máquinas para el posterior procesado de la información.

Para reconocer la voz del contenedor del audio, este se extrae en trocitos o chunks que se tratan posteriormente para sacar un texto plano.

En nuestro caso usaremos **Vosk** ya que es muy efectivo y los modelos son muy ligeros.

| Nombre      | Vosk                                                                                                                                                                                                                                                                             |
|-------------|-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| Licencia    | Apache License 2.0                                                                                                                                                                                                                                                                                 |
| Descripción | Vosk es un conjunto de herramientas de reconocimiento de voz con una API de streaming para la mejor experiencia de usuario. También hay bindings para diferentes lenguajes de programación: java/csharp/javascript, etc. Permite una rápida reconfiguración del vocabulario para obtener la mejor precisión.|

Los modelos de Vosk se pueden extraer de [este link](https://alphacephei.com/vosk/models) y para probarlo solo hay que extraer el ZIP y llamar en el código a la carpeta del modelo.

In [3]:
#!/usr/bin/env python3
# Script de reconocimiento.

from vosk import Model, KaldiRecognizer
import sys
import json

model = Model(model_path='model')

# Large vocabulary free form recognition
rec = KaldiRecognizer(model, 16000)

wf = open('file.wav', "rb")
wf.read(44) # skip header

while True:
    data = wf.read(4000)
    if len(data) == 0:
        break
    if rec.AcceptWaveform(data):
        res = json.loads(rec.Result())
        print('Partial')
        print (res['text'])

res = json.loads(rec.FinalResult())
print('Total')
print (res['text'])

Total
hola buenos días


## 3. Text-to-Speech. Convirtiendo el texto en voz.

Si el reconocimiento de voz trata el procesamiento del habla para extraer un texto, la síntesis de voz sería lo contrario, la producción artificial de ese habla.
Para ello se usan como base los grafemas (aquellas letras y grupos que se pronuncian de una manera). De esa forma, se separa el texto en dichos grafemas, tras lo cual se asocian esos grafemas a sus correspondientes sonidos o fonemas asociados, entonando así cada palabra, frase y finalmente leyendo el texto.

En este caso usaremos [eSpeakNG](https://github.com/espeak-ng/espeak-ng), por su facilidad de uso. Se puede instalar fácilmente en Linux y Windows (y en MacOS se está trabajando en un port). Por contra, la voz no es muy natural.

| Nombre      | eSpeakNG                                                                                                                                                                                                                                                                             |
|-------------|-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| Licencia    | Apache License 2.0                                                                                                                                                                                                                                                                                 |
| Descripción | |

¡Ojo! Para usarse en Python sigue los pasos que indican en el paquete de pip (https://pypi.org/project/espeakng/)



In [5]:
# Script de síntesis de voz
from espeakng import Speaker

esng = Speaker()

esng.wpm = 140
esng.pitch = 120
esng.voice = 'es'
esng.say("Hola mundo, ¿qué tal estáis?")

## 4. Chatbots y cómo nos responde una IA.



### 4.1 Creando modelos en Rasa

Para crear un modelo en Rasa invocaremos el siguiente comando.

```bash
rasa init
```

### 4.2 Entrenamiento y pruebas

Para entrenar el modelo en Rasa invocaremos el siguiente comando.
```bash
rasa train
```

Para probarlo, invocaremos este otro comando.
```bash
rasa shell
```

### 4.3 Conectando el servidor de Rasa.

Para poner a Rasa en modo de servidor, podremos lanzar este comando:

```bash
rasa run
```

In [ ]:
import requests
sender = input("What is your name?\n")

bot_message = ""
while bot_message != "Adiós":
    message = input("What's your message?\n")

    print("Sending message now...")

    r = requests.post('http://localhost:5005/webhooks/rest/webhook', json={"sender": sender, "message": message})

    print(r.json())

    print("Bot says, ")
    for i in r.json():
        try:
            bot_message = i['text']
        except KeyError:
            try:
                bot_message = i['image']
            except KeyError:
                bot_message = ''
        print(f"{bot_message}")

### 4.4 Funciones custom

Las funciones custom se pueden lanzar usando:

```bash
rasa run actions
```

In [ ]:
# En el archivo chatbot/actions/actions.py ...

from rasa_sdk import Action, Tracker
from rasa_sdk.executor import CollectingDispatcher
import requests

class ActionTellTime(Action):
    """
    Clase para la acción de decir la hora
    """

    def name(self) -> Text:
        """
        Declaración de la acción
        @return string Nombre de la acción
        """
        return "action_tell_time"

    def run(self, dispatcher: CollectingDispatcher,
            tracker: Tracker,
            domain: Dict[Text, Any]) -> List[Dict[Text, Any]]:
        """
        Ejecución de la acción.
        Extrae el datetime de este instante y extrae la hora
        @utter_message Devuelve la hora en formato HH y MM (15:30 -> Son las 15 horas y 30 minutos)
        """
        dispatcher.utter_message(text=f"Son las \
            {datetime.datetime.now().strftime('%H horas y %M minutos')}")

        return []